In [1]:
import numpy as np
import pandas as pd
import os
import torch
from datetime import datetime

ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
current_path = os.getcwd()
case_name = 'basecase'
today = datetime.today().strftime("%y%m%d")
print("Today is", today, "\nCurrent path is", current_path)

Today is 230310 
Current path is c:\Joko\Project\Bigdata\randomforest


In [13]:
"""
!!!!!!!!!! 필수 체크 !!!!!!!!!!

RF 모델은 LSTM 결과와 비교를 위해 만들어졌으므로, LSTM에서 만들어진 데이터셋(Train/Valid/Test)을 그대로 가져와서 가공하는 형식으로 코드를 만들었음.

RF 용 데이터로 쪼개야함. LSTM 모델에서 이미 나눠진 데이터를 가지고 와서 내가 원하는 데이터 형태로 바꿀 수 없을까?
Target Features: TVD / MD / (주어진 기간에서) Peak Production rate 까지 걸린 개월 수 / Maximum Peak Production Rate / Cumulative Produciton rate / Re-fracking Information / EUR (= Cumulative production rate ~ 36개월)
TVD, MD를 제외하고 모두 이미 만들어진 파일로 사용 가능. 그러면 TVD / MD 를 Completion에서 매칭해서 가져오는 코드를 작성하자. --> 이러면 나중에 내가 원하는 변수를 따로 가져올 수 있음.

Step 1: 하나의 파일을 불러와서, API number 로 Completion 파일에서 원하는 변수를 찾고 이를 새로운 열로 만드는 것. seq 길이가 안맞는건 어떻게 해결? 안되면 나머진 다 NULL 값 처리
Step 2: 가져온 파일들을 바탕으로 동일 이름의 새로운 Randomforest 용 하나의 dataframe 생성. 즉 Train 용 / Valid용 / Test용 dataset 존재
"""
input_path_fornamelst = current_path + '/data/basecase_from_LSTM/'
namelst = os.listdir(input_path_fornamelst)

if not os.path.exists(current_path + '\\data'):
        os.makedirs(current_path + '\\data')

if not os.path.exists(current_path + '\\data\\' + case_name):
        os.makedirs(current_path + '\\data\\' + case_name)

if not os.path.exists(current_path + '\\data\\' + case_name + '\\case1'):
        os.makedirs(current_path + '\\data\\' + case_name + '\\case1')

input_path_fornamelst_final = current_path + '/data/basecase_from_LSTM/case1'
namelst_final = os.listdir(input_path_fornamelst_final)

In [27]:
#### Control Pannel ####
extrac_lst = ['TVD_FT', 'MD_FT'] # completion data 열 이름과 반드시 동일해야함.
used_period_lst = [3, 6, 12, 24, 36]
maximum_used_period = used_period_lst[-1]

#### Control Pannel ####

# for dirname, _, filenames in os.walk(current_path + input_path_LSTM): ## 한번에 불러오는 건 담에 하자.
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


source_path = current_path + '/source/completion/'
source_lst = os.listdir(source_path)
source_lst
for source in source_lst:
    source_comp = pd.read_csv(source_path + source)
    df_comp = pd.DataFrame(source_comp)

for directory in namelst_final:

    input_path = current_path + '/data/basecase_from_LSTM/case1/' + directory + '/'
    input_lst = os.listdir(input_path)

    prod_rate_lst = []
    for mon in used_period_lst: ## Peak prod rate 값
        prod_rate_lst = np.append(prod_rate_lst, 'peak_prod_used_' + str(mon) + 'mon')
    for mon in used_period_lst: ## Peak prod rate일 때의 월
        prod_rate_lst = np.append(prod_rate_lst, 'time_to_peak_used_' + str(mon) + 'mon')
    for mon in range(maximum_used_period): ## 월별 생산량
        prod_rate_lst = np.append(prod_rate_lst, 'prod_rate_' + str(mon+1) + 'mon')
    for mon in range(maximum_used_period): ## 주어진 기간 동안 cum 생산량
        prod_rate_lst = np.append(prod_rate_lst, 'cum_rate_' + str(mon) + 'mon')
    prod_rate_lst = np.append(prod_rate_lst, 'EUR') ## EUR 값 = Target = Label

    tmp_lst = np.concatenate((extrac_lst, prod_rate_lst))
    default_lst = ['API', 'shutin', 'refracking', 'produced_month']
    column_lst = np.concatenate((default_lst, tmp_lst))

    df = pd.DataFrame(columns=column_lst)

    forsetting_lst = [] ## Data를 하나씩 넣어주기 위해 우선 열 길이에 맞는 nan 값을 생성해서 할당하기 위한 lst
    for _ in range(len(column_lst)):
        forsetting_lst = np.append(forsetting_lst, 'NaN')

    ## 반복문 시작 ##

    for file in input_lst:
        source_train = pd.read_csv(input_path + file)
        df_LSTM = pd.DataFrame(source_train)
        df_LSTM.head()
        df_LSTM['cum_gas_rate'] = 0

        EUR_standard = len(df_LSTM)

        for idx in range(len(df_LSTM)):
            if idx == 0:
                df_LSTM['cum_gas_rate'][idx] = df_LSTM['Gas_rate'][idx]
            else:
                df_LSTM['cum_gas_rate'][idx] = df_LSTM['Gas_rate'][idx] + df_LSTM['cum_gas_rate'][idx-1]

        filename = file.strip('.csv')

        df_tmp = pd.DataFrame(columns=column_lst)
        df_tmp.loc[0] = forsetting_lst
        df_tmp['API'] = filename

        if df_LSTM['ShutinMonths'].sum() == 0:
            df_tmp['shutin'] = 0
        else: df_tmp['shutin'] = 1

        if df_LSTM['Refrac'].sum() == 0:
            df_tmp['refracking'] = 0
        else: df_tmp['refracking'] = 1

        df_tmp['produced_month'] = EUR_standard ## 총 생산일수를 EUR 기준으로 넣어줌

        for ext in extrac_lst: # completion source에서 원하는 정보 뽑는 코드 !!! 주의 !!! 여기서 같은 API에 서로 다른 내용이 여러개 있을 수 있는데 그럴 경우 일괄적으로 그냥 맨 위에 값 썼음. 특별한 이유는 X. 다른 방법이 없어서임.
            df_tmp[ext] = df_comp[df_comp['API_UWI'] == filename][ext].values[0]

        for period in used_period_lst:
            df_tmp['peak_prod_used_' + str(period) + 'mon'] = df_LSTM.loc[df_LSTM['Gas_rate'][:period].idxmax()]['Gas_rate']
            df_tmp['time_to_peak_used_' + str(period) + 'mon'] = df_LSTM.loc[df_LSTM['Gas_rate'][:period].idxmax()]['TotalProdMonths']

        for mon in range(maximum_used_period): ## 월별 생산량
            df_tmp['prod_rate_' + str(mon+1) + 'mon'] = df_LSTM['Gas_rate'][mon]
            df_tmp['cum_rate_' + str(mon+1) + 'mon'] = df_LSTM['cum_gas_rate'][mon]

        df_tmp['EUR'] = df_LSTM['cum_gas_rate'][EUR_standard-1]

        df = pd.concat([df, df_tmp])

    df.to_csv(current_path + '/data/' + case_name + '/case1/' + directory + '.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_17896\1260753443.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_LSTM['cum_gas_rate'][idx] = df_LSTM['Gas_rate'][idx]
C:\Users\User\AppData\Local\Temp\ipykernel_17896\1260753443.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_LSTM['cum_gas_rate'][idx] = df_LSTM['Gas_rate'][idx]
C:\Users\User\AppData\Local\Temp\ipykernel_17896\1260753443.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [28]:
input_path = current_path + '/data/basecase/case1/'
train_source = pd.read_csv(input_path + 'Train.csv')
df_train = pd.DataFrame(train_source)
display(df_train.head())

test_source = pd.read_csv(input_path + 'Test.csv')
df_test = pd.DataFrame(test_source)
display(df_test.head())

,Unnamed: 0,API,shutin,refracking,produced_month,TVD_FT,MD_FT,peak_prod_used_3mon,peak_prod_used_6mon,peak_prod_used_12mon,...,cum_rate_28mon,cum_rate_29mon,cum_rate_30mon,cum_rate_31mon,cum_rate_32mon,cum_rate_33mon,cum_rate_34mon,cum_rate_35mon,EUR,cum_rate_36mon
0,0,30-015-20871,1,1,237,9286.0,11668.0,44.766667,270.258065,270.258065,...,4692.862372,4925.462372,5166.545706,5550.212372,6039.649872,6286.972453,6513.553098,6746.410241,52633.457620,6959.861854
1,0,30-015-21529,1,0,77,13544.0,13544.0,2896.652174,2896.652174,2896.652174,...,34092.358114,34846.896576,35687.753719,36472.463396,37231.930063,37978.381676,38725.307602,39365.210827,65120.906948,39933.533408
2,0,30-015-22627,1,1,180,13117.0,13117.0,1850.733333,1850.733333,1850.733333,...,13599.290666,13734.129375,13910.833079,14068.220176,14221.020176,14382.020176,14512.981714,14668.981714,72699.447793,14856.046230
3,0,30-015-23131,1,0,201,12485.0,12485.0,3506.800000,3533.774194,3533.774194,...,61287.909050,62928.650985,64523.258128,66009.548451,67523.848451,68982.751676,70335.885010,71403.078558,127384.050887,72574.691461
4,0,30-015-25033,0,0,214,12222.0,12226.0,941.806452,941.806452,941.806452,...,23083.599002,23773.179647,24486.953840,25226.953840,25927.599002,26645.232335,27360.071045,28026.804378,87607.547733,28740.965668


,Unnamed: 0,API,shutin,refracking,produced_month,TVD_FT,MD_FT,peak_prod_used_3mon,peak_prod_used_6mon,peak_prod_used_12mon,...,cum_rate_28mon,cum_rate_29mon,cum_rate_30mon,cum_rate_31mon,cum_rate_32mon,cum_rate_33mon,cum_rate_34mon,cum_rate_35mon,EUR,cum_rate_36mon
0,0,30-015-31937,1,0,147,12280.0,12280.0,4351.571429,4728.322581,4851.709677,...,71642.755897,72629.497832,73187.336542,73685.869875,74283.353746,75161.687080,75842.429015,76685.848370,88703.767676,77498.562656
1,0,30-015-32116,1,0,211,7996.0,8000.0,2392.066667,2507.096774,2717.107143,...,46518.637576,47455.378317,49000.344983,50323.594983,51460.728317,52760.308962,54061.728317,55335.621174,98744.270623,56536.814722
2,0,30-015-32181,1,0,96,7992.0,9850.0,1003.148148,1208.933333,1450.233333,...,20572.570703,20832.109165,21160.823450,21649.758934,21779.706303,21791.480496,21813.880496,21869.525657,23159.166865,21904.389294
3,0,30-015-32298,1,0,225,12230.0,12230.0,1747.838710,1747.838710,1747.838710,...,24522.650901,24968.554127,25461.287460,25931.610041,26291.739073,26841.560502,27358.528244,27866.328244,43546.658458,28322.392760
4,0,30-015-32340,1,0,215,11757.0,11757.0,631.733333,631.733333,631.733333,...,9333.624184,9583.979023,9824.979023,10083.656442,10347.256442,10591.417733,10830.772571,11080.405905,42801.923833,11311.857518


In [30]:
########################### Start of Control Pannel #########################
used_period_lst         = [3, 6, 12, 24, 36]  ## 리스트 형태로 들어갈 경우 각 요소별로 모델을 따로 만들어서 결과를 확인함

EUR_standard            = 'Length of each wells' ## 사실 이건 못 바꿈. 이거 바꾸려면 데이터셋 설정부터 다시해야함

use_or_not_shutin            = 1
use_or_not_refracking        = 1
use_or_not_TVD               = 0
use_or_not_MD                = 0
use_or_not_produced_month    = 1

use_or_not_peak_prod    = 1
use_or_not_time_to_peak = 1
use_or_not_prod_rate    = 1
use_or_not_cum_rate     = 0


########################### End of Control Pannel ###########################


In [31]:
## Train part

from sklearn.ensemble import RandomForestRegressor

for used_period in used_period_lst:
    input_fetures = []
    if use_or_not_shutin == 1:
        input_fetures = np.append(input_fetures, 'shutin')
    if use_or_not_refracking == 1:
        input_fetures = np.append(input_fetures, 'refracking')
    if use_or_not_produced_month == 1:
        input_fetures = np.append(input_fetures, 'produced_month')
    if use_or_not_TVD == 1:
        input_fetures = np.append(input_fetures, 'TVD_FT')
    if use_or_not_MD == 1:
        input_fetures = np.append(input_fetures, 'MD_FT')

    if use_or_not_peak_prod == 1:
        input_fetures = np.append(input_fetures, 'peak_prod_used_' + str(used_period) + 'mon')
    if use_or_not_time_to_peak == 1:
        input_fetures = np.append(input_fetures, 'time_to_peak_used_' + str(used_period) + 'mon')

    if use_or_not_prod_rate == 1:
        for mon in range(used_period):
            input_fetures = np.append(input_fetures, 'prod_rate_' + str(mon+1) + 'mon')
    if use_or_not_cum_rate == 1:
        for mon in range(used_period):
            input_fetures = np.append(input_fetures, 'cum_rate_' + str(mon+1) + 'mon')


    print('input fetatures are', input_fetures)
    x_train = df_train[input_fetures]
    y_train = df_train['EUR']

    x_test = df_test[input_fetures]
    y_test = df_test['EUR']

    print()
    print('x_train shape is', x_train.shape, 'y_train shape is', y_train.shape)
    print('x_test shape is', x_test.shape, 'y_test shape is', y_test.shape)

    RF_model = RandomForestRegressor(n_estimators=100, random_state=0)
    x_train = x_train.fillna(0)
    RF_model.fit(x_train, y_train) ## 일단 NaN 값을 0으로 처리하자. NaN이 있는 데이터를 Drop 할 수도 있지만.

    x_test = x_test.fillna(0)
    y_pred = RF_model.predict(x_test)

    colums_EUR = ['name', 'True', 'Pred']
    df_EUR = pd.DataFrame(columns=colums_EUR)
    df_EUR['name'] = df_test['API'].values
    df_EUR['True'] = y_test
    df_EUR['Pred'] = y_pred
    df_EUR.to_csv(current_path + '/results/' + case_name + '/EUR_scatterplot_' + str(used_period) + '.csv')

input fetatures are ['shutin' 'refracking' 'produced_month' 'peak_prod_used_3mon'
 'time_to_peak_used_3mon' 'prod_rate_1mon' 'prod_rate_2mon'
 'prod_rate_3mon']

x_train shape is (2283, 8) y_train shape is (2283,)
x_test shape is (405, 8) y_test shape is (405,)
input fetatures are ['shutin' 'refracking' 'produced_month' 'peak_prod_used_6mon'
 'time_to_peak_used_6mon' 'prod_rate_1mon' 'prod_rate_2mon'
 'prod_rate_3mon' 'prod_rate_4mon' 'prod_rate_5mon' 'prod_rate_6mon']

x_train shape is (2283, 11) y_train shape is (2283,)
x_test shape is (405, 11) y_test shape is (405,)
input fetatures are ['shutin' 'refracking' 'produced_month' 'peak_prod_used_12mon'
 'time_to_peak_used_12mon' 'prod_rate_1mon' 'prod_rate_2mon'
 'prod_rate_3mon' 'prod_rate_4mon' 'prod_rate_5mon' 'prod_rate_6mon'
 'prod_rate_7mon' 'prod_rate_8mon' 'prod_rate_9mon' 'prod_rate_10mon'
 'prod_rate_11mon' 'prod_rate_12mon']

x_train shape is (2283, 17) y_train shape is (2283,)
x_test shape is (405, 17) y_test shape is (405,)

In [ ]:
"""
이부분은 나중을 위한 부분
"""

from sklearn.model_selection import train_test_split
import category_encoders as ce

### categorical variable 사용 시 예시 코드 ordinalEncoder 말고 Onehotcoding 등도 있음 함수는 찾아봐
# encoder = ce.OrdinalEncoder(cols=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])


# X_train = encoder.fit_transform(X_train)

# X_test = encoder.transform(X_test)